In [1]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-7.4506e-09, grad_fn=<MeanBackward0>)

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.2526, -0.1947,  0.5426],
        [-0.4186, -0.8788, -0.7419],
        [ 0.2094, -0.0429,  0.5749],
        [ 0.6335,  1.0470,  1.7426],
        [-0.0394,  0.6063, -0.9017]], requires_grad=True)

In [7]:
linear(torch.rand(2, 5))

tensor([[1.0277, 0.9016, 0.3455],
        [0.9336, 0.5464, 0.6713]])

In [8]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[ 4.4887],
        [10.7497]])

In [ ]:
class BilinearReduction(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.W = nn.Parameter(torch.randn(output_dim, input_dim, input_dim))
        
    def forward(self, x):
        # x shape: (batch_size, input_dim)
        # 计算 y_k = einsum('bi, kij, bj -> bk', x, W, x)
        return torch.einsum('bi, kij, bj -> bk', x, self.W, x)

layer = BilinearReduction(input_dim=10, output_dim=5)
x = torch.randn(3, 10)  # 批量大小=3，输入维度=10
y = layer(x)             # 输出形状：(3, 5)

In [10]:
y.shape 

torch.Size([3, 5])

In [ ]:
class FourierCoeffLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        fft_complex = torch.fft.rfft(x, dim=1)  # 实数FFT，输出形状：(batch_size, seq_len//2+1)
        return fft_complex

layer = FourierCoeffLayer()
x = torch.randn(3, 100)        # 批量大小=3，序列长度=100
coeffs = layer(x)              # 输出形状：(3, 51)
coeffs.shape

torch.Size([3, 51])